### Import related package

In [1]:
## Import tensorflow package for modeling
import tensorflow as tf

## Data processing
import pandas as pd
import numpy as np

## Min-max normalization
from sklearn.preprocessing import MinMaxScaler

## Plot the graph
import matplotlib.pyplot as plt
%matplotlib inline

## Initializing module
from sklearn.linear_model import LinearRegression
np.set_printoptions(suppress=True)

## Copy module
import copy

## Used to calculate the training time
import time

## Set the GUP environment
import os

### Control memory usage space for GPU

In [2]:
# gpus = tf.config.list_physical_devices(device_type='GPU')

# tf.config.set_logical_device_configuration(
#     gpus[0],
#     [tf.config.LogicalDeviceConfiguration(memory_limit=2048),
#      tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
# )

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction =0.5
tf.compat.v1.Session(config=config)

In [5]:
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# tf.compat.v1.keras.backend.set_session(sess)

### Preprocessing the data

In [6]:
def read(path):
    return pd.read_csv(path)

In [7]:
def buildTrain(train, pastWeek=4, futureWeek=4, defaultWeek=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureWeek-pastWeek):
        X = np.array(train.iloc[i:i+defaultWeek])
        X = np.append(X,train["CCSP"].iloc[i+defaultWeek:i+pastWeek])
        X_train.append(X.reshape(X.size))
        Y_train.append(np.array(train.iloc[i+pastWeek:i+pastWeek+futureWeek]["CCSP"]))
    return np.array(X_train), np.array(Y_train)

### Min-max normalization

In [8]:
## Use min-max normalization to scale the data to the range from 1 to 0
sc = MinMaxScaler(feature_range = (0, 1))

### Design get_data() to get data

In [9]:
def get_data(futureWeek):
    
    ## Read weekly copper price data
    path = "WeeklyFinalData.csv"
    data = read(path)
    
    date = data["Date"]
    data.drop("Date", axis=1, inplace=True)
    
    ## Add time lag (pastWeek=4, futureWeek=1)
    x_data, y_data = buildTrain(data, futureWeek=futureWeek)
    
    x_data1 = x_data[:157]
    y_data1 = y_data[:157]
    ## Split the data to training data and test data
    x_train_data = x_data1[:int(x_data1.shape[0]*0.8)]
    x_test_data = x_data1[int(x_data1.shape[0]*0.8):]
    y_train_data = y_data1[:int(x_data1.shape[0]*0.8)]
    y_test_data = y_data1[int(x_data1.shape[0]*0.8):]


    return (x_train_data, x_test_data, y_train_data, y_test_data)

#     return (x_data, y_data)

In [10]:
x_train_data, x_test_data, y_train_data, y_test_data = get_data(4)
print(x_train_data.shape)
print(x_test_data.shape)
print(y_train_data.shape)
print(y_test_data.shape)

(125, 18)
(32, 18)
(125, 4)
(32, 4)


### Network class

In [11]:
class Network():
    
    def __init__(self, nb_neuro, x_train_scaled, y_train_scaled):
        
        # Stop criteria - threshold
        self.threshold_for_error = 0.07
        self.threshold_for_lr = 1e-6
        
        # Input data 
        self.x = tf.convert_to_tensor(x_train_scaled, np.float32)
        self.y = tf.convert_to_tensor(y_train_scaled, np.float32)
        
        # Learning rate
        self.learning_rate = 1e-2
        
        
        # Hidden layer I
        self.n_neurons_in_h1 = nb_neuro
        self.W1 = tf.Variable(tf.random.truncated_normal([self.x.shape[1], self.n_neurons_in_h1], mean=0, stddev=1))
        self.b1 = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1], mean=0, stddev=1))

        # Output layer
        self.Wo = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1, self.y.shape[1]], mean=0, stddev=1))
        self.bo = tf.Variable(tf.random.truncated_normal([self.y.shape[1]], mean=0, stddev=1))

        # Whether the network is acceptable, default as False
        self.acceptable = False
        
        # Some record for experiment
        self.nb_node_pruned = 0
        self.nb_node_acceptable=tf.convert_to_tensor([nb_neuro])
        
    
    ## Forecast the test data
    def forecast(self, x_test_scaled):
    
        x_test_scaled = tf.cast(x_test_scaled, tf.float32)
        activation_value = tf.nn.relu((tf.matmul(x_test_scaled, self.W1)+self.b1))
        forecast_value = tf.matmul(activation_value,self.Wo)+self.bo
       
        return forecast_value

    ## Reset the x and y data
    def setData(self, x_train_scaled, y_train_scaled):
        self.x = tf.convert_to_tensor(x_train_scaled, np.float32)
        self.y = tf.convert_to_tensor(y_train_scaled, np.float32)
    
    ## Add the new data to the x and y data
    def addData(self, new_x_train, new_y_train):

        self.x = tf.concat([self.x, new_x_train.reshape(1,-1)],0)
        self.y = tf.concat([self.y, new_y_train.reshape(1,-1)],0)
    
    ## forward operation
    def forward(self,  reg_strength= 0):
        with tf.GradientTape() as tape:

            y1 = tf.nn.relu((tf.matmul(self.x, self.W1)+self.b1))
            yo = (tf.matmul(y1,self.Wo)+self.bo)

            # performance measure
            diff = yo-self.y
            loss = tf.reduce_mean(diff**2) + (reg_strength/(self.Wo.shape[1]*(self.Wo.shape[0]+1)+self.W1.shape[1]*(self.W1.shape[0]+1))) * ((tf.nn.l2_loss(self.W1) + tf.nn.l2_loss(self.Wo) + tf.nn.l2_loss(self.b1) + tf.nn.l2_loss(self.bo))*2)

        return(yo, loss, tape)

    # backward operation
    def backward_Adam(self,tape,loss):

        optimizer = tf.optimizers.Adam(self.learning_rate)
        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])
        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))
    
    def backward_RMS(self,tape,loss):

        optimizer = tf.keras.optimizers.RMSprop(self.learning_rate)
        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])
        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))

### Initializing module (Check)

In [12]:
def initializing(network, initial_x, initial_y):
    
    ## Find each minimum output value y
    min_y = tf.reduce_min(initial_y, axis=0)
    
    ## Subtract min_y from each y
    res_y = initial_y-min_y
    
    ## Use linear regression to find the initial W1,b1,Wo,bo
    reg = LinearRegression().fit(initial_x, res_y)

    ## Set up the initial parameter of the network
    network.W1 = tf.Variable(tf.cast(tf.transpose(reg.coef_), tf.float32))
    network.b1 = tf.Variable(tf.convert_to_tensor(reg.intercept_, tf.float32))
    network.Wo = tf.Variable(tf.convert_to_tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], tf.float32))
    network.bo = tf.Variable(tf.cast(min_y, tf.float32))

    ## Set up the acceptable of the initial network as True
    network.acceptable =True

### Selecting module (Check)

In [13]:
def selecting(network, x_train_scaled, y_train_scaled):
    
    
    loss = []
    temp_network = copy.deepcopy(network)
    
    ## Put each data into network to calculate the loss value
    for i in range(x_train_scaled.shape[0]):
        temp_network.setData(x_train_scaled[i].reshape(1,-1), y_train_scaled[i].reshape(1,-1))
        loss.append((temp_network.forward()[1].numpy(),i))

    ## Sort the data according to the loss value from smallest to largest, and save the data index in sorted_index
    sorted_index = [sorted_data[1] for sorted_data in sorted(loss, key = lambda x:x[0])]

    ## Print out some info for debug
    print("The loss value of k:",loss[sorted_index[0]])
    print("Selecting module finish!")
    
    return sorted_index

### Matching module (Check)

In [14]:
def matching(network):

    ## Set up the learning rate of the network
    network.learning_rate = 1e-3
    

    while True:
        
        ## Get the loss value of the current network architecture
        yo, loss, tape = network.forward()

        ## Identify that all forecast value has met the error term
        if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
            
            ## If true, set the acceptable of the network as true and return it
            network.acceptable = True
            print("Matching finished - the network is acceptable")
            return(network)


        ## If the error is not satisfied, continue to tunning the learning rate of the network
        else:
            
            # Save the current papameter
            network_pre = copy.deepcopy(network)
            
            # Stroe the last loss value
            loss_pre = loss
            
            # Backward and check the loss performance of the network with new learning rate
            network.backward_Adam(tape,loss)
            yo, loss, tape = network.forward()

            # Confirm whether the loss value of the adjusted network is smaller than the current one
            if loss < loss_pre:

                # If true, multiply the learning rate by 1.2
                network.learning_rate *= 1.2

            # On the contrary, reduce the learning rate
            else:         
                
                # Identify whether the current learning rate is less than the threshold
                if network.learning_rate <= network.threshold_for_lr:
                    
                    # If true, set the acceptable of the network as false and return it
                    network.acceptable = False
                    print("Matching finished - the network is Unacceptable")
                    return(network)

                # On the contrary, restore w and adjust the learning rate
                else:
                    
                    # Restore the papameter of the network
                    network = network_pre
                    network.learning_rate *= 0.7

### Cramming module (Check)

In [15]:
def cramming(network):
    

    ## Set the random seed
    tf.random.set_seed(5)

    ## Find unsatisfied data:K
    yo, loss, tape = network.forward()
    undesired_index = tf.where(tf.math.abs(yo-network.y) > network.threshold_for_error)

    ## Print out the undesired_index for debug
    print("The index of the undesired data:",undesired_index)


    ## Unsatisfied situation
    for i in range(undesired_index.shape[0]):

        ## Find the index of the unsatisfied data
        k_data_num = undesired_index[i][0]
        k_l = undesired_index[i][1]

        undesired_data = tf.reshape(network.x[k_data_num,:], [1,-1])

        ## Remove the data that does not meet the error term
        left_data = network.x[:k_data_num,:]
        right_data = network.x[k_data_num+1:,:]
        remain_tensor = tf.concat([left_data, right_data], 0)

        
        ## Use the random method to find out the gamma and zeta
        while True:

            ## Find m-vector gamma: r
            ## Use the random method to generate the gamma that can make the conditions met
            gamma = tf.random.uniform(shape=[1,network.x.shape[1]])
            subtract_undesired_data = tf.subtract(remain_tensor, undesired_data)
            matmul_value = tf.matmul(gamma,tf.transpose(subtract_undesired_data))


            if tf.reduce_all(matmul_value != 0):

                while True:

                    ## Find the tiny value: zeta
                    ## Use the random method to generate the zeta that can make the conditions met
                    zeta = tf.random.uniform(shape=[1])

                    if tf.reduce_all(tf.multiply(tf.add(zeta,matmul_value),tf.subtract(zeta,matmul_value))<0):
                        break

                break


        ## The weight of input layer to hidden layer I
        w10 = gamma
        w11 = gamma
        w12 = gamma
        
        W1_new = tf.transpose(tf.concat([w10,w11,w12],0))

        ## The bias of input layer to hidden layer I
        matual_value = tf.matmul(gamma,tf.transpose(undesired_data))

        b10 = tf.subtract(zeta,matual_value)
        b11 = -1*matual_value
        b12 = tf.subtract(-1*zeta,matual_value)
        b1_new = tf.reshape(tf.concat([b10,b11,b12],0),[3])

        ## The weight of hidden layer I to output layer
        gap = network.y[k_data_num, k_l]-yo[k_data_num, k_l]

        wo0_value = gap/zeta
        wo1_value = (-2*gap)/zeta
        wo2_value = gap/zeta

        wo0 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo0_value, shape=(1,-1))
        wo1 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo1_value, shape=(1,-1))
        wo2 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo2_value, shape=(1,-1))

        Wo_new = tf.concat([wo0,wo1,wo2],0)

        ## Add new neuroes to the network
        network.W1 = tf.Variable(tf.concat([network.W1, W1_new],1), tf.float32)
        network.b1 = tf.Variable(tf.concat([network.b1, b1_new],0), tf.float32)
        network.Wo = tf.Variable(tf.concat([network.Wo, Wo_new],0), tf.float32)

        yo, loss, tape = network.forward()
   
        ## Determine if cramming is successful and print out the corresponding information
        if tf.reduce_all(tf.math.abs(yo[k_data_num,k_l]-network.y[k_data_num,k_l]) <= network.threshold_for_error):
            
            ## If the cramming process is complete, set the acceptable of the network as true
            if i==(undesired_index.shape[0]-1):
                network.acceptable = True
            
            print("Cramming success!")
            
        
        else:
            print("Cramming failed!")

### Regularizing module (Check)

In [16]:
def regularizing(network):

    ## Record the number of executions
    times_enlarge = 0
    times_shrink = 0
    ## Set up the learning rate of the network
    network.learning_rate = 1e-3

    ## Set epoch to 100
    for i in range(100):

        ## Store the parameter of the network
        network_pre = copy.deepcopy(network)
        yo, loss, tape = network.forward(1e-2)
        loss_pre = loss

        ## Backward operation to obtain w'
        network.backward_Adam(tape, loss)
        yo, loss, tape = network.forward(1e-2)

         # Confirm whether the adjusted loss value is smaller than the current one
        if loss <= loss_pre:
            
            ## Identify that all forecast value has met the error term
            if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
                
                ## If true, multiply the learning rate by 1.2
                network.learning_rate *= 1.2
                times_enlarge += 1
#                 print("Regularizing process")

            else:

                ## Else, restore w and end the process
                network = network_pre
                print("Number of enlarge:",times_enlarge)
                print("Number of shrink:",times_shrink)
                print("Regularizing result: Unable to meet the error term")
                return(network)

        # If the adjusted loss value is not smaller than the current one
        else:
           

            ## If the learning rate is greater than the threshold for learning rate
            if network.learning_rate > network.threshold_for_lr:
                
                ## Restore the w and multiply the learning rate by 0.7
                network = network_pre
                network.learning_rate *= 0.7
                times_shrink += 1
             ## If the learning rate is smaller than the threshold for learning rate
            else:
                
                ## Restore the w
                network = network_pre
                print("Number of enlarge:",times_enlarge)
                print("Number of shrink:",times_shrink)
                print("Regularizing result: Less than the epsilon for the learning rate")
                return(network)

        if i == 99:
            print("Number of enlarge:",times_enlarge)
            print("Number of shrink:",times_shrink)
            print("Regularizing result: The number of rounds has reached")
            return(network)

### Reorganizing module (Check)

In [17]:
def reorganizing(network):
    
    ## Set up the k = 1, and p = the number of hidden node
    k = 1
    p = network.W1.shape[1]

    while True:

        ## If k > p, end of Process
        if k > p:

            print("Reorganizing result: The final number of neuro is ",p)
            return(network)

        ## Else, Process is ongoing
        else:

            ## Using the regularizing module to adjust the network
            network = regularizing(network)
            
            ## Store the network and w
            network_pre = copy.deepcopy(network)

            ## Set up the acceptable of the network as false
            network.acceptable = False

            ## Ignore the K hidden node
            network.W1 = tf.Variable(tf.concat([network.W1[:,:k-1],network.W1[:,k:]],1))
            network.b1 = tf.Variable(tf.concat([network.b1[:k-1],network.b1[k:]],0))
            network.Wo = tf.Variable(tf.concat([network.Wo[:k-1,:],network.Wo[k:,:]],0))

            ## Using the matching module to adjust the network
            network = matching(network)

            ## If the resulting network is acceptable, this means that the k hidden node can be removed
            if network.acceptable:

                print("Drop out the nero number: %d / %d" %(k, p))
                network.nb_node_pruned += 1
                ## p--
                p-=1

            ## Else, it means that the k hidden node cannot be removed
            else:
                
                ## Restore the network and w
                network = network_pre
                print("Cannot drop out the nero number: %d / %d" %(k, p))
                
                ## k++
                k+=1
                    

In [18]:
## Call the help method "get_data(...)" to get the training data and test data 
x_train_data, x_test_data, y_train_data, y_test_data = get_data(4)

## Use min-max normalization to normalize data in a range of 0 to 1
x_train_scaled = sc.fit_transform(x_train_data)
x_test_scaled = sc.transform(x_test_data)
y_train_scaled = sc.fit_transform(y_train_data)

## Use the first 25 training data to debug
x_train_scaled = x_train_scaled[:22]
y_train_scaled = y_train_scaled[:22]

## Pick up m+1 data that are linearly independent as the initial m+1 training data 
initial_x = x_train_scaled[:x_train_scaled.shape[1]+1]
initial_y = y_train_scaled[:x_train_scaled.shape[1]+1]

## Construct the network object with 4 neuros
network = Network(4, initial_x, initial_y)

In [19]:
network.b1.shape[0]

4

### Construct a instance of network
- trained through the matching module, reorganizing module, and cramming module

In [20]:
start = time.time()
nb_step4 = 0
nb_step6_1 = 0
nb_step6_2 = 0

## Call the help method "get_data(...)" to get the training data and test data 
x_train_data, x_test_data, y_train_data, y_test_data = get_data(4)

## Use min-max normalization to normalize data in a range of 0 to 1
x_train_scaled = sc.fit_transform(x_train_data)
x_test_scaled = sc.transform(x_test_data)
y_train_scaled = sc.fit_transform(y_train_data)

## Use the first 25 training data to debug
# x_train_scaled = x_train_scaled[:22]
# y_train_scaled = y_train_scaled[:22]

## Pick up m+1 data that are linearly independent as the initial m+1 training data 
initial_x = x_train_scaled[:x_train_scaled.shape[1]+1]
initial_y = y_train_scaled[:x_train_scaled.shape[1]+1]

## The remaining data (that exclude initial data)
x_train_scaled = x_train_scaled[x_train_scaled.shape[1]+1:]
y_train_scaled = y_train_scaled[x_train_scaled.shape[1]+1:]

## Construct the network object with 4 neuros
network = Network(4, initial_x, initial_y)

## Use initilize module to set up the initial network
initializing(network, initial_x, initial_y)


## Training of all data
for i in range(0, x_train_scaled.shape[0]):
    
   
    ## Print out some info for debug
    print("The data index: %d"%(i+x_train_scaled.shape[1]+2))
    
    ## Get the data index by selecting module to sort the data by the loss value from smallest to largest
    sorted_index = selecting(network, x_train_scaled, y_train_scaled)
    
    ## Add new data for training
    network.addData(x_train_scaled[sorted_index[0]], y_train_scaled[sorted_index[0]])
    x_train_scaled = np.delete(x_train_scaled, sorted_index[0], 0)
    y_train_scaled = np.delete(y_train_scaled, sorted_index[0], 0)
    
    yo, loss, tape = network.forward()
    
    ## Print out some information for debug
    print("The error term for each data")
    print(tf.math.abs(yo-network.y))
    
    ## Determine whether the forecast value can meet the error term
    if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
        
        ## If true, set up the acceptable of the network as true
        network.acceptable = True
        
        ## Use reorganizing module to adjust the model
        network = reorganizing(network)
        
        ## Record the number of runs
        nb_step4 += 1
 
    else:
        
        ## If true, set up the acceptable of the network as false
        network.acceptable = False
        network_pre = copy.deepcopy(network)
        
        ## Use matching module to adjust the model
        network = matching(network)
        
        ## If the output of the matching module is an acceptable network, use the reorganization module to adjust the model
        if network.acceptable:
            network = reorganizing(network)
            
            ## Record the number of runs
            nb_step6_1 += 1
 
        ## Else (if the output of the matching module is an unacceptable network)
        else:
            network = network_pre
            
            ## Use cramming module and reorganizing module to adjust the model
            cramming(network)
            network = reorganizing(network)
            
            ## Record the number of runs
            nb_step6_2 += 1

    ## Print out the model status
    network.nb_node_acceptable = tf.concat([network.nb_node_acceptable, [network.b1.shape[0]]],0)
    print("The network status:",network.acceptable)
    print("-"*10,"next data","-"*10)
## Calculate the training time    
end = time.time()

The data index: 20
The loss value of k: (0.12238025, 0)
Selecting module finish!
The error term for each data
tf.Tensor(
[[0.00000125 0.0000059  0.         0.00000048]
 [0.00000054 0.00000352 0.00000036 0.00000042]
 [0.00000471 0.00000358 0.00000024 0.00000012]
 [0.         0.00000161 0.00000036 0.00000036]
 [0.00000149 0.00000274 0.00000054 0.0000006 ]
 [0.00000036 0.00000083 0.00000024 0.0000006 ]
 [0.00000107 0.00000346 0.00000006 0.00000024]
 [0.00000012 0.00000107 0.00000024 0.00000036]
 [0.00000131 0.00000048 0.00000006 0.00000054]
 [0.00000215 0.00000179 0.00000012 0.00000077]
 [0.00000203 0.00000077 0.0000003  0.00000036]
 [0.00000054 0.00000089 0.00000024 0.00000018]
 [0.00000113 0.0000006  0.00000018 0.00000042]
 [0.00000036 0.00000042 0.00000054 0.00000054]
 [0.00000364 0.00000018 0.00000054 0.0000003 ]
 [0.00000578 0.0000003  0.0000003  0.00000036]
 [0.00000006 0.00000328 0.00000012 0.00000048]
 [0.0000003  0.00000799 0.00000012 0.00000012]
 [0.00000441 0.0000031  0.0000002

ResourceExhaustedError: OOM when allocating tensor with shape[109,491] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Relu]

In [ ]:
forecast_value = network.forecast(x_test_scaled)

y_pred = sc.inverse_transform(forecast_value)
accuracy = list()

for i in range(y_pred.shape[1]):
#     for _ in range(y_pred.shape[0]): 

    correct_times = np.sum(tf.math.abs(y_test_data[:,i]-y_pred[:,i])<2000)
    accuracy.append(correct_times/y_pred.shape[0])

In [ ]:
print("The training time(s):",end - start)

print("-"*30)
total_time = nb_step4 + nb_step6_1 + nb_step6_2
print("The percentage of each step")
print("Step 4:%.2f"%(nb_step4/total_time))
print("Step 6.1:%.2f"%(nb_step6_1/total_time))
print("Step 6.2:%.2f"%(nb_step6_2/total_time))

print("-"*30)
print("Total amount of cramming occurrences:",nb_step6_2)

print("-"*30)
print("The amount of hidden node that be pruned:",network.nb_node_pruned)

print("-"*30)
print("The amount of adopted hidden nodes:",network.nb_node_acceptable[-1].numpy())

print("-"*30)
print("Accuracy")
print("The accuracy for the t+1:",accuracy[0])
print("The accuracy for the t+2:",accuracy[1])
print("The accuracy for the t+3:",accuracy[2])
print("The accuracy for the t+4:",accuracy[3])

In [1]:
yo, loss, tape = network.forward()
plt.plot(sc.inverse_transform(yo)[:,0], label="LLAAT")
plt.plot(sc.inverse_transform(network.y)[:,0], label="Actual")
plt.legend()

NameError: name 'network' is not defined

In [ ]:
# forecast_value = network.forecast(x_test_scaled)
plt.plot(sc.inverse_transform(forecast_value)[:,0], label="LLAAT")
plt.plot(y_test_data[:,0], label="Actual")
plt.legend()